In [4]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from statsmodels.stats.multitest import multipletests


# ---- config ----
excel_path = r"Z:\UriMons\cFos\Quantification_Results\correlations_template.xlsx"  # <- replace with your file path
sheet_name = "r_and_N"

# ---- helpers ----
def fisher_z(r):
    # clip r slightly to avoid inf at |r|=1
    r = np.clip(r, -0.999999, 0.999999)
    return np.arctanh(r)

def se_from_n(n):
    return 1.0 / np.sqrt(np.maximum(n - 3, 1e-9))  # guard for small N

def z_test_independent(z1, se1, z2, se2):
    # z of difference between two independent Fisher z's
    denom = np.sqrt(se1**2 + se2**2)
    z = (z1 - z2) / denom
    # two-tailed p-value
    from scipy.stats import norm
    p = 2 * (1 - norm.cdf(np.abs(z)))
    return z, p

def fisher_ci_r(r, n, alpha=0.05):
    """95% CI for r via Fisher z, returns (lo, hi) on r scale"""
    r = np.clip(float(r), -0.999999, 0.999999)
    z = np.arctanh(r)
    se = 1.0 / np.sqrt(max(n - 3, 1e-9))
    zcrit = norm.ppf(1 - alpha/2)
    lo_z, hi_z = z - zcrit*se, z + zcrit*se
    return np.tanh(lo_z), np.tanh(hi_z)

# ---- load ----
df = pd.read_excel(excel_path, sheet_name=sheet_name)

# expected columns
cols = [
    "Region",
    "WT_control_r","WT_control_N",
    "WT_stress_r","WT_stress_N",
    "KO_control_r","KO_control_N",
    "KO_stress_r","KO_stress_N",
]
missing = [c for c in cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# ---- compute Fisher z, SE, and 95% CI for each r ----
groups = ["WT_control","WT_stress","KO_control","KO_stress"]
for g in groups:
    r_col, n_col = f"{g}_r", f"{g}_N"
    df[f"{g}_z"]  = fisher_z(df[r_col])
    df[f"{g}_SE"] = se_from_n(df[n_col])

    # 95% CI for r (on r scale)
    lo_list, hi_list = [], []
    for r, n in zip(df[r_col].to_numpy(), df[n_col].to_numpy()):
        lo, hi = fisher_ci_r(r, n, alpha=0.05)
        lo_list.append(lo); hi_list.append(hi)
    df[f"{g}_r_CI95_lo"] = lo_list
    df[f"{g}_r_CI95_hi"] = hi_list

# ---- comparisons (independent samples) ----
def add_comp(df, a, b, name):
    # z-test
    zstat, p = z_test_independent(df[f"{a}_z"], df[f"{a}_SE"],
                                  df[f"{b}_z"], df[f"{b}_SE"])
    df[f"{name}_zstat"] = zstat
    df[f"{name}_p"] = p
    # Δr on raw correlation scale (direction matches zstat numerator: a - b)
    df[f"{name}_delta_r"] = df[f"{a}_r"] - df[f"{b}_r"]

# within-genotype stress effect
add_comp(df, "WT_stress", "WT_control", "WT_stress_vs_control")
add_comp(df, "KO_stress", "KO_control", "KO_stress_vs_control")
# between-genotype within each condition
add_comp(df, "KO_stress", "WT_stress", "Stress_KO_vs_WT")
add_comp(df, "KO_control", "WT_control", "Control_KO_vs_WT")

# ---- FDR per contrast family ----
for base in ["WT_stress_vs_control", "KO_stress_vs_control",
             "Stress_KO_vs_WT", "Control_KO_vs_WT"]:
    pvals = df[f"{base}_p"].to_numpy()
    _, qvals, _, _ = multipletests(pvals, method="fdr_bh")
    df[f"{base}_q"] = qvals

# ---- save ----
out_path = excel_path.replace(".xlsx", "_with_fisherZ_tests_and_deltaR.xlsx")
df.to_excel(out_path, index=False)
print(f"Saved: {out_path}")

Saved: Z:\UriMons\cFos\Quantification_Results\correlations_template_with_fisherZ_tests_and_deltaR.xlsx
